# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120732e+02     1.548328e+00
 * time: 0.734083890914917
     1     1.051803e+01     8.411327e-01
 * time: 2.0006399154663086
     2    -1.216414e+01     9.948632e-01
 * time: 2.676654815673828
     3    -3.421532e+01     7.160164e-01
 * time: 3.6419880390167236
     4    -4.792789e+01     4.853440e-01
 * time: 4.611682891845703
     5    -5.692338e+01     1.623754e-01
 * time: 5.5798180103302
     6    -5.972911e+01     1.261209e-01
 * time: 6.222069025039673
     7    -6.085427e+01     5.247603e-02
 * time: 6.872222900390625
     8    -6.130150e+01     6.389584e-02
 * time: 7.514561891555786
     9    -6.161698e+01     3.883126e-02
 * time: 8.176854848861694
    10    -6.184311e+01     2.564056e-02
 * time: 8.822357892990112
    11    -6.200856e+01     2.101894e-02
 * time: 9.46508502960205
    12    -6.207294e+01     2.046114e-02
 * time: 10.110251903533936
    13    -6.214013e+01     1.359340e-02
 * time: 10.754849910736084
    14

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557663
    AtomicNonlocal      14.8522633
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336820

    total               -62.261666460493
